# Book Rental Recommendation

### Import all lib

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## Load all data

In [2]:
book_rating = pd.read_csv("BX-Book-Ratings.csv")
book = pd.read_csv("BX-Books.csv")
user = pd.read_csv("BX-Users.csv")

C:\Users\subha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\subha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
book_rating

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6
...,...,...,...
1048570,250764,451410777,0
1048571,250764,452264464,8
1048572,250764,048623715X,0
1048573,250764,486256588,0


In [4]:
book

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271374,440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271375,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271377,192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [5]:
user

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278854,278854,"portland, oregon, usa",NaN
278855,278855,"tacoma, washington, united kingdom",50.0
278856,278856,"brampton, ontario, canada",NaN
278857,278857,"knoxville, tennessee, usa",NaN


### Preprocessing

In [6]:
book.isnull().sum()

isbn                   0
book_title             0
book_author            1
year_of_publication    0
publisher              2
dtype: int64

In [7]:
book = book.dropna(axis = 0)

In [8]:
book.isnull().sum()

isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
dtype: int64

In [9]:
print("Unique values = ", book_rating["user_id"].unique())
print("Total unique value = ", len(book_rating["user_id"].unique()))

Unique values =  [276725 276726 276727 ... 250760 250761 250764]
Total unique value =  95513


In [10]:
print("Unique values = ",book_rating["isbn"].unique())
print("Total unique value = ",len(book_rating["isbn"].unique()))

Unique values =  ['034545104X' '155061224' '446520802' ... '048623715X' '486256588'
 '515069434']
Total unique value =  322102


In [11]:
book_rating = book_rating.merge(book, on="isbn")

### For getting the recommendation of book user must be familiar to model, so usser need to rate x_n number of book and we can recommend only those book which is rated by y_n user

In [12]:
x_n = 250   #for this model we take x_n= 250 and y_n = 100
y_n = 100

In [13]:
x = book_rating.groupby(by = "user_id").count()["rating"]>x_n
y = x[x].index
genuine_usser_rating = book_rating[book_rating["user_id"].isin(y)]

In [14]:
genuine_usser_rating

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
15,77940,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
16,81977,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
...,...,...,...,...,...,...,...
940956,250405,1862071713,0,"Remind Me Who I Am, Again",Linda Grant,2000,Pub Group West
940957,250405,1879557487,0,Evolution &amp; Extinction: Essays,Stephen Jay Gould,1998,Audio Scholar
940958,250405,188424405X,8,"Menopause, Naturally: Preparing for the Second...",Sadja Greenwood,1996,Volcano Press
940959,250405,1884433847,0,Life Launch: A Passionate Guide to the Rest of...,Frederic M. Hudson,2000,Hudson Pr


In [15]:
x = book_rating.groupby(by = "isbn").count()["rating"]>y_n
y = x[x].index
genuine_rating = genuine_usser_rating[genuine_usser_rating["isbn"].isin(y)]

In [16]:
genuine_rating

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
56,278418,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
58,3363,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
59,7158,446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books
62,11676,446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books
67,23768,446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books
...,...,...,...,...,...,...,...
477600,242083,312971125,6,Crazy for You,Jennifer Crusie,2000,St. Martin's Press
477601,242106,312971125,0,Crazy for You,Jennifer Crusie,2000,St. Martin's Press
477602,242299,312971125,0,Crazy for You,Jennifer Crusie,2000,St. Martin's Press
477604,244688,312971125,0,Crazy for You,Jennifer Crusie,2000,St. Martin's Press


# user-based collaborative filtering

In [17]:
piv = genuine_rating.pivot_table(values="rating",index="user_id",columns="isbn")

In [18]:
piv.fillna(0,inplace=True)
piv

isbn,002542730X,006016848X,006099486X,006101351X,014023313X,014025448X,014028009X,014029628X,014100018X,031242227X,...,805063897,812511816,812550706,842329129,842329218,842329242,842342702,871136791,971880107,99771519
user_id,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277427,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
distance = cosine_similarity(piv)          # Calculating cosine distance of each user from each user

In [20]:
distance.shape

(553, 553)

In [21]:
def list_enu(w):
    list =[]
    for i in w:
        list.append(i)
    return list

## Recommendetor 
Take user id as input and print name of 5 book 

In [22]:
def recommendetor(x):
    index = np.where(piv.index==x)[0][0]
    closest_user = sorted(list_enu(enumerate(distance[index])),key = lambda x:x[1])[-2][0]
    most_ratted_book_index = genuine_rating[genuine_rating["user_id"]==piv.index[closest_user]]["rating"].sort_values(ascending=False)[:5].index
    for i in most_ratted_book_index:
        print(book_rating.iloc[i,3])   

In [23]:
recommendetor(254)    #Demo

Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
She's Come Undone (Oprah's Book Club)


# item-based collaborative filtering

In [24]:
piv1 = genuine_rating.pivot_table(values="rating",index="book_title",columns="user_id")

In [25]:
piv1.fillna(0,inplace=True)
piv1

user_id,254,2276,2766,3363,4385,6251,6543,6575,7158,7346,...,247752,248718,249628,249862,249894,250184,250405,277427,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
distance1 = cosine_similarity(piv1)

# Recommendetor
Take Book name as input and print 5 book as recommendation

In [27]:
def recommendetor(x):
    index = np.where(piv1.index==x)[0][0]
    closest_book = sorted(list_enu(enumerate(distance1[index])),key = lambda x:x[1])[1:6]
    for i in closest_book:
        print(piv1.index[i[0]])

In [28]:
recommendetor("You Belong To Me")   # demo

A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash
A Civil Action
A Heartbreaking Work of Staggering Genius
A Thousand Acres (Ballantine Reader's Circle)
A Virtuous Woman (Oprah's Book Club (Paperback))
